In [ ]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from modules._import_helper import safe_import_from

# Import utilities and KF
set_seed = safe_import_from('00_repo_standards.src.mlphys_core', 'set_seed')
(KalmanFilter, constant_velocity_model, position_observation_model) = safe_import_from(
    '04_time_series_state_space.src.kalman',
    'KalmanFilter', 'constant_velocity_model', 'position_observation_model'
)

set_seed(42)
plt.style.use('default')

# Create output directory
output_dir = Path('modules/04_time_series_state_space/reports/nb02_kalman_filter')
output_dir.mkdir(parents=True, exist_ok=True)

print("✓ Setup complete")

## 3. Generate Ground Truth Data

Reuse simulation from Notebook 01.

In [ ]:
def simulate_tracking_system(dt, n_steps, process_noise_std, obs_noise_std, seed=42):
    """Simulate constant-velocity tracking system."""
    rng = np.random.default_rng(seed)
    
    F, Q = constant_velocity_model(dt, process_noise_std)
    H, R = position_observation_model(obs_noise_std)
    
    true_states = []
    observations = []
    x = np.array([0.0, 1.0])  # Start: position=0, velocity=1 m/s
    
    for _ in range(n_steps):
        w = rng.multivariate_normal(np.zeros(2), Q)
        x = F @ x + w
        true_states.append(x.copy())
        
        v = rng.normal(0, obs_noise_std)
        z = H @ x + v
        observations.append(z[0])
    
    times = np.arange(n_steps) * dt
    return times, np.array(true_states), np.array(observations), F, H, Q, R

# Baseline parameters
dt = 0.1
n_steps = 100
process_noise = 0.02
obs_noise = 0.15

times, true_states, observations, F, H, Q, R = simulate_tracking_system(
    dt, n_steps, process_noise, obs_noise, seed=42
)

print(f"Generated {n_steps} timesteps")
print(f"System: dt={dt}s, σ_w={process_noise}, σ_v={obs_noise}")

## 4. Run Kalman Filter

Apply KF from `src/kalman.py` to the noisy observations.

In [ ]:
# Initialize Kalman Filter
kf = KalmanFilter(F, H, Q, R)

# Initial guess: assume stationary at origin with high uncertainty
x0 = np.array([0.0, 0.0])
P0 = np.eye(2) * 1.0  # High initial uncertainty
kf.initialize(x0, P0)

# Run filter
estimated_states = []
covariances = []
innovations = []

for z in observations:
    # Predict
    kf.predict()
    
    # Update
    innovation = kf.update(np.array([z]))
    
    # Store
    x_est, P_est = kf.get_state()
    estimated_states.append(x_est)
    covariances.append(P_est)
    innovations.append(innovation[0])

estimated_states = np.array(estimated_states)
covariances = np.array(covariances)
innovations = np.array(innovations)

print(f"✓ Kalman filter applied to {n_steps} measurements")

## 5. Visualize Results

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 10))

# Position tracking
ax = axes[0]
ax.plot(times, true_states[:, 0], 'b-', linewidth=2, label='True Position')
ax.scatter(times, observations, c='gray', s=15, alpha=0.5, label='Noisy Measurements', zorder=1)
ax.plot(times, estimated_states[:, 0], 'r-', linewidth=2, label='KF Estimate', zorder=2)

# Uncertainty bands (±2σ)
pos_std = np.sqrt(covariances[:, 0, 0])
ax.fill_between(times, 
                 estimated_states[:, 0] - 2*pos_std,
                 estimated_states[:, 0] + 2*pos_std,
                 color='red', alpha=0.2, label='95% Confidence')

ax.set_ylabel('Position (m)', fontsize=12)
ax.legend(loc='upper left', fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_title('Kalman Filter: Position Tracking', fontsize=13)

# Velocity tracking
ax = axes[1]
ax.plot(times, true_states[:, 1], 'g-', linewidth=2, label='True Velocity')
ax.plot(times, estimated_states[:, 1], 'r-', linewidth=2, label='KF Estimate')

vel_std = np.sqrt(covariances[:, 1, 1])
ax.fill_between(times,
                 estimated_states[:, 1] - 2*vel_std,
                 estimated_states[:, 1] + 2*vel_std,
                 color='red', alpha=0.2, label='95% Confidence')

ax.set_ylabel('Velocity (m/s)', fontsize=12)
ax.legend(loc='upper left', fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_title('Velocity Estimation (unobserved state!)', fontsize=13)

# Innovations (should be zero-mean if filter is consistent)
ax = axes[2]
ax.plot(times, innovations, 'k-', linewidth=1, alpha=0.7, label='Innovations')
ax.axhline(0, color='red', linestyle='--', linewidth=2, label='Expected mean')
ax.fill_between(times, -2*obs_noise, 2*obs_noise, color='red', alpha=0.1, label='±2σ_v')
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Innovation', fontsize=12)
ax.legend(loc='upper left', fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_title('Innovation Sequence (should be white noise)', fontsize=13)

plt.tight_layout()
plt.savefig(output_dir / 'kf_tracking_results.png', dpi=120, bbox_inches='tight')
plt.show()

print("\n💡 Observations:")
print("   - KF smooths noisy measurements")
print("   - Estimates velocity despite no direct measurement!")
print("   - Uncertainty bands shrink as filter converges")

## 6. Performance Metrics

In [ ]:
# Compute RMSE
def rmse(true, estimated):
    return np.sqrt(np.mean((true - estimated)**2))

rmse_pos = rmse(true_states[:, 0], estimated_states[:, 0])
rmse_vel = rmse(true_states[:, 1], estimated_states[:, 1])

# Observation RMSE (no filtering)
rmse_obs = rmse(true_states[:, 0], observations)

print("\n" + "="*50)
print("PERFORMANCE METRICS")
print("="*50)
print(f"Position RMSE (observations): {rmse_obs:.4f} m")
print(f"Position RMSE (KF estimate):  {rmse_pos:.4f} m")
print(f"Velocity RMSE (KF estimate):  {rmse_vel:.4f} m/s")
print("="*50)

improvement = (rmse_obs - rmse_pos) / rmse_obs * 100
print(f"\n✨ KF improves position estimate by {improvement:.1f}%")

# Innovation statistics (consistency check)
innov_mean = np.mean(innovations)
innov_std = np.std(innovations)

print(f"\nInnovation statistics:")
print(f"  Mean: {innov_mean:.4f} (should be ~0)")
print(f"  Std:  {innov_std:.4f} (should be ~{obs_noise:.4f})")

if abs(innov_mean) < 0.05 and abs(innov_std - obs_noise) < 0.05:
    print("  ✓ Filter is consistent!")
else:
    print("  ⚠️ Filter may be inconsistent (check Q/R tuning)")

## 7. Covariance Evolution

Watch how uncertainty changes over time.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Position uncertainty
ax = axes[0]
pos_var = covariances[:, 0, 0]
ax.plot(times, np.sqrt(pos_var), 'b-', linewidth=2)
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Position Std Dev (m)', fontsize=12)
ax.set_title('Position Uncertainty Over Time', fontsize=13)
ax.grid(True, alpha=0.3)

# Velocity uncertainty
ax = axes[1]
vel_var = covariances[:, 1, 1]
ax.plot(times, np.sqrt(vel_var), 'g-', linewidth=2)
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Velocity Std Dev (m/s)', fontsize=12)
ax.set_title('Velocity Uncertainty Over Time', fontsize=13)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'covariance_evolution.png', dpi=120, bbox_inches='tight')
plt.show()

print("\n💡 Covariance converges to steady-state after ~10-20 steps")
print("   This is typical for time-invariant linear systems.")

## 8. Experiment: Noise Sensitivity

**Question:** How does KF performance vary with noise levels?

Sweep both process and measurement noise.

In [ ]:
# Parameter sweep
process_noise_values = [0.01, 0.02, 0.05, 0.1, 0.2]
obs_noise_values = [0.05, 0.1, 0.15, 0.3, 0.5]

results = []

for pn in process_noise_values:
    for on in obs_noise_values:
        # Simulate
        times, true_states, observations, F, H, Q, R = simulate_tracking_system(
            dt, n_steps, pn, on, seed=42
        )
        
        # Run KF
        kf = KalmanFilter(F, H, Q, R)
        kf.initialize(np.array([0, 0]), np.eye(2))
        
        estimated = []
        for z in observations:
            kf.predict()
            kf.update(np.array([z]))
            x_est, _ = kf.get_state()
            estimated.append(x_est)
        
        estimated = np.array(estimated)
        
        # Metrics
        rmse_pos = rmse(true_states[:, 0], estimated[:, 0])
        rmse_vel = rmse(true_states[:, 1], estimated[:, 1])
        
        results.append({
            "σ_w": pn,
            "σ_v": on,
            "RMSE_pos": rmse_pos,
            "RMSE_vel": rmse_vel,
        })

import pandas as pd
df_sweep = pd.DataFrame(results)

print("\n" + "="*60)
print("NOISE SENSITIVITY SWEEP (sample)")
print("="*60)
print(df_sweep.head(10).to_string(index=False))
print("...")
print("="*60)

# Save full table
df_sweep.to_csv(output_dir / 'noise_sweep_results.csv', index=False)
print(f"\n✓ Full sweep saved to {output_dir / 'noise_sweep_results.csv'}")

### Heatmap: RMSE vs Noise Parameters

In [ ]:
# Reshape for heatmap
rmse_matrix = df_sweep.pivot(index='σ_w', columns='σ_v', values='RMSE_pos')

fig, ax = plt.subplots(figsize=(10, 7))
im = ax.imshow(rmse_matrix.values, cmap='viridis', aspect='auto', origin='lower')

# Labels
ax.set_xticks(range(len(obs_noise_values)))
ax.set_xticklabels([f"{v:.2f}" for v in obs_noise_values])
ax.set_yticks(range(len(process_noise_values)))
ax.set_yticklabels([f"{v:.2f}" for v in process_noise_values])

ax.set_xlabel('Measurement Noise σ_v', fontsize=12)
ax.set_ylabel('Process Noise σ_w', fontsize=12)
ax.set_title('KF Position RMSE vs Noise Parameters', fontsize=13)

# Colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('RMSE (m)', fontsize=12)

# Annotate cells
for i in range(len(process_noise_values)):
    for j in range(len(obs_noise_values)):
        text = ax.text(j, i, f"{rmse_matrix.values[i, j]:.3f}",
                      ha="center", va="center", color="white", fontsize=9)

plt.tight_layout()
plt.savefig(output_dir / 'rmse_heatmap.png', dpi=120, bbox_inches='tight')
plt.show()

print("\n💡 Observations:")
print("   - High measurement noise → higher RMSE (can't see true state well)")
print("   - High process noise → higher RMSE (system is unpredictable)")
print("   - Diagonal trend: both matter!")

## 9. Common Pitfalls

### Pitfall 1: Wrong Q/R Scaling

What happens if we **underestimate** measurement noise (R too small)?

In [ ]:
# Generate data with obs_noise = 0.3
times, true_states, observations, F, H, Q_true, R_true = simulate_tracking_system(
    dt, n_steps, process_noise=0.02, obs_noise_std=0.3, seed=42
)

# Run KF with WRONG R (too optimistic)
R_wrong = np.array([[0.05**2]])  # Assume σ_v = 0.05 when actually 0.3

kf_wrong = KalmanFilter(F, H, Q_true, R_wrong)
kf_wrong.initialize(np.array([0, 0]), np.eye(2))

estimated_wrong = []
for z in observations:
    kf_wrong.predict()
    kf_wrong.update(np.array([z]))
    x_est, _ = kf_wrong.get_state()
    estimated_wrong.append(x_est)

estimated_wrong = np.array(estimated_wrong)

# Compare with correct R
kf_correct = KalmanFilter(F, H, Q_true, R_true)
kf_correct.initialize(np.array([0, 0]), np.eye(2))

estimated_correct = []
for z in observations:
    kf_correct.predict()
    kf_correct.update(np.array([z]))
    x_est, _ = kf_correct.get_state()
    estimated_correct.append(x_est)

estimated_correct = np.array(estimated_correct)

# Plot
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(times, true_states[:, 0], 'b-', linewidth=2, label='True', zorder=3)
ax.scatter(times, observations, c='gray', s=15, alpha=0.4, label='Observations')
ax.plot(times, estimated_correct[:, 0], 'g-', linewidth=2, label='KF (correct R)', zorder=2)
ax.plot(times, estimated_wrong[:, 0], 'r--', linewidth=2, label='KF (wrong R, too small)', zorder=2)

ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Position (m)', fontsize=12)
ax.set_title('Pitfall: Underestimating Measurement Noise', fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'pitfall_wrong_R.png', dpi=120, bbox_inches='tight')
plt.show()

rmse_correct = rmse(true_states[:, 0], estimated_correct[:, 0])
rmse_wrong = rmse(true_states[:, 0], estimated_wrong[:, 0])

print(f"\n⚠️ Effect of wrong R:")
print(f"   RMSE (correct R): {rmse_correct:.4f} m")
print(f"   RMSE (wrong R):   {rmse_wrong:.4f} m")
print(f"\n   Filter with wrong R over-trusts noisy measurements!")
print(f"   Result: Estimate is too 'jittery' and tracks noise.")

### Pitfall 2: Divergence from Wrong Initialization

What if initial state estimate is very wrong and P0 is too small?

In [ ]:
# Same data
times, true_states, observations, F, H, Q, R = simulate_tracking_system(
    dt, n_steps, process_noise=0.02, obs_noise_std=0.15, seed=42
)

# Bad initialization: x0 very wrong, P0 too confident
kf_bad = KalmanFilter(F, H, Q, R)
x0_bad = np.array([10.0, -5.0])  # Very wrong initial guess
P0_bad = np.eye(2) * 0.01  # Overconfident (too small)
kf_bad.initialize(x0_bad, P0_bad)

estimated_bad = []
for z in observations:
    kf_bad.predict()
    kf_bad.update(np.array([z]))
    x_est, _ = kf_bad.get_state()
    estimated_bad.append(x_est)

estimated_bad = np.array(estimated_bad)

# Good initialization
kf_good = KalmanFilter(F, H, Q, R)
kf_good.initialize(np.array([0, 0]), np.eye(2))  # High uncertainty

estimated_good = []
for z in observations:
    kf_good.predict()
    kf_good.update(np.array([z]))
    x_est, _ = kf_good.get_state()
    estimated_good.append(x_est)

estimated_good = np.array(estimated_good)

# Plot
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(times, true_states[:, 0], 'b-', linewidth=2, label='True')
ax.plot(times, estimated_good[:, 0], 'g-', linewidth=2, label='Good init (P0=I)')
ax.plot(times, estimated_bad[:, 0], 'r--', linewidth=2, label='Bad init (x0 wrong, P0 small)')

ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Position (m)', fontsize=12)
ax.set_title('Pitfall: Bad Initialization', fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'pitfall_initialization.png', dpi=120, bbox_inches='tight')
plt.show()

print(f"\n⚠️ Bad initialization:")
print(f"   Filter takes many steps to converge if P0 is too small.")
print(f"   Solution: Use large P0 if initial state is uncertain!")

## 10. Key Takeaways

✅ **Kalman Filter** is optimal for linear-Gaussian systems  
✅ **Two-step recursion**: Predict (dynamics) → Update (measurement)  
✅ **Kalman gain** automatically balances prediction vs measurement  
✅ **Covariance** tracks uncertainty (converges to steady-state)  
✅ **Innovations** should be zero-mean white noise (consistency check)  
✅ **Q and R tuning** is critical: wrong values → poor performance  
✅ **Initialization**: Use large P0 if state is uncertain  

---

## 11. Exercises

### Exercise 1: Manual Prediction Step

**Task:** Given state estimate $\hat{x} = [1.0, 0.5]^T$ and $P = I$, compute the predicted state and covariance after one predict step.

Use: $F = \begin{bmatrix} 1 & 0.1 \\ 0 & 1 \end{bmatrix}$, $Q = 0.02^2 \cdot I$

In [ ]:
# Your code here

### Exercise 2: Tune Q and R

**Task:** For the baseline system (σ_w=0.02, σ_v=0.15), try:
1. Increase Q by 10x
2. Increase R by 10x
3. Increase both by 10x

Compare RMSE. Which has more impact?

In [ ]:
# Your code here

### Exercise 3: Innovation Analysis

**Task:** Compute the autocorrelation of innovations (lag 1, 2, 3). If filter is consistent, autocorrelation should be near zero.

Hint: Use `np.corrcoef(innovations[:-k], innovations[k:])` for lag k.

In [ ]:
# Your code here

---

## 12. Solutions

### Solution 1: Manual Prediction

In [ ]:
# Solution
x_hat = np.array([1.0, 0.5])
P = np.eye(2)
F = np.array([[1, 0.1], [0, 1]])
Q = np.eye(2) * (0.02**2)

# Predict
x_pred = F @ x_hat
P_pred = F @ P @ F.T + Q

print("Predicted state:")
print(x_pred)
print("\nPredicted covariance:")
print(P_pred)

### Solution 2: Tune Q and R

In [ ]:
# Solution
times, true_states, observations, F, H, Q_base, R_base = simulate_tracking_system(
    dt, n_steps, 0.02, 0.15, seed=42
)

configs = [
    ("Baseline", Q_base, R_base),
    ("Q x10", Q_base * 10, R_base),
    ("R x10", Q_base, R_base * 10),
    ("Both x10", Q_base * 10, R_base * 10),
]

results_tuning = []
for name, Q, R in configs:
    kf = KalmanFilter(F, H, Q, R)
    kf.initialize(np.array([0, 0]), np.eye(2))
    
    estimated = []
    for z in observations:
        kf.predict()
        kf.update(np.array([z]))
        x_est, _ = kf.get_state()
        estimated.append(x_est)
    
    estimated = np.array(estimated)
    rmse_pos = rmse(true_states[:, 0], estimated[:, 0])
    results_tuning.append({"Config": name, "RMSE": rmse_pos})

df_tuning = pd.DataFrame(results_tuning)
print("\n" + "="*40)
print("Q/R Tuning Impact")
print("="*40)
print(df_tuning.to_string(index=False))
print("="*40)

print("\n💡 Increasing R (measurement noise) has more impact on RMSE.")
print("   This is because measurements provide most information.")

### Solution 3: Innovation Autocorrelation

In [ ]:
# Solution
# Run KF to get innovations (already have from earlier)
# Compute autocorrelation
lags = [1, 2, 3, 5, 10]
autocorr = []

for lag in lags:
    if lag < len(innovations):
        corr = np.corrcoef(innovations[:-lag], innovations[lag:])[0, 1]
        autocorr.append(corr)
    else:
        autocorr.append(np.nan)

plt.figure(figsize=(10, 5))
plt.stem(lags, autocorr, basefmt=' ')
plt.axhline(0, color='red', linestyle='--', linewidth=2)
plt.axhline(0.2, color='gray', linestyle=':', alpha=0.5, label='±0.2 threshold')
plt.axhline(-0.2, color='gray', linestyle=':', alpha=0.5)
plt.xlabel('Lag', fontsize=12)
plt.ylabel('Autocorrelation', fontsize=12)
plt.title('Innovation Autocorrelation (should be near 0)', fontsize=13)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(output_dir / 'ex3_innovation_autocorr.png', dpi=120, bbox_inches='tight')
plt.show()

print("\n💡 Low autocorrelation → innovations are white noise → filter is consistent!")

---

## Summary Report

In [ ]:
# Write summary
summary = f"""
# Notebook 02: Kalman Filter - Summary

**Date:** {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}

## Key Results

1. **Baseline KF Performance:**
   - Position RMSE improved from {rmse_obs:.4f} to {rmse_pos:.4f} m
   - Improvement: {improvement:.1f}%

2. **Noise Sensitivity:**
   - Tested {len(process_noise_values)} x {len(obs_noise_values)} = {len(results)} configurations
   - Measurement noise has stronger impact than process noise

3. **Common Pitfalls:**
   - Wrong R → over/under-trusting measurements
   - Bad initialization (P0 too small) → slow convergence

## Outputs Generated
   - kf_tracking_results.png
   - covariance_evolution.png
   - rmse_heatmap.png
   - noise_sweep_results.csv

## Next Steps

→ Notebook 03: Nonlinear systems (EKF, Particle Filter)
"""

with open(output_dir / 'summary.md', 'w') as f:
    f.write(summary)

print("\n" + "="*60)
print("✓ Notebook 02 Complete!")
print("="*60)
print(f"Outputs saved to: {output_dir}")
print("\nFiles created:")
for f in sorted(output_dir.glob('*')):
    print(f"  - {f.name}")